# K7 Meta-Hodge v1.0

Offline analysis pipeline that mines all historical K7/G2 PINN runs to
assemble improved Hodge operators, harmonic bases, and Yukawa couplings.


In [ ]:
%load_ext autoreload
%autoreload 2
import torch
from pathlib import Path
from G2_ML.meta_hodge import (
    CandidateLibrary,
    HarmonicSolver,
    HodgeOperator,
    locate_historical_assets,
    load_version_model,
    sample_coords,
    YukawaExtractor,
)

assets = locate_historical_assets()
assets


In [ ]:
# Choose a reference metric (v1.8 assumed to be the most stable).
reference_version = '1_8' if '1_8' in assets else list(assets)[0]
bundle = load_version_model(reference_version, assets)
x = sample_coords(4000)
g = bundle.metric_fn(x)
phi = bundle.phi_fn(x) if bundle.phi_fn else None
g.shape, None if phi is None else phi.shape


In [ ]:
# Collect candidate forms from *all* discovered versions.
library = CandidateLibrary(versions=list(assets))
candidates = library.collect(x)
candidates.metadata


In [ ]:
# Build Hodge operators and solve for harmonic representatives.
op = HodgeOperator(g)
solver = HarmonicSolver(op)
harmonic = solver.solve(candidates.c2, candidates.c3, k2=21, k3=77)
harmonic['H2'].eigenvalues[:5], harmonic['H3'].eigenvalues[:5]


In [ ]:
# Compute Yukawa couplings via Monte Carlo.
extractor = YukawaExtractor(x, g)
yukawa = extractor.compute(harmonic['H2'].eigenvectors, harmonic['H3'].eigenvectors)
paths = extractor.save(yukawa, Path('artifacts/meta_hodge'))
paths
